In [2]:

import speech_recognition as sr
import pyttsx3
import pywhatkit
import datetime
import wikipedia
import pyjokes
import requests
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

# Initialize recognizer and engine
listener = sr.Recognizer()
engine = pyttsx3.init()
voices = engine.getProperty('voices')
engine.setProperty('voice', voices[1].id)

def talk(text):
    """Speak the provided text."""
    engine.say(text)
    engine.runAndWait()

def take_command():
    """Listen for a command from the user."""
    command = None
    try:
        with sr.Microphone() as source:
            print('Listening...')
            listener.adjust_for_ambient_noise(source)
            voice = listener.listen(source, timeout=10, phrase_time_limit=5)
            command = listener.recognize_google(voice)
            command = command.lower()
            if 'alexa' in command:
                command = command.replace('alexa', '').strip()
                print(f"Command: {command}")
            else:
                talk("Please start your command with 'Alexa'.")
                return None
    except sr.UnknownValueError:
        talk("Sorry, I did not understand that.")
    except sr.RequestError as e:
        talk(f"Sorry, my speech service is down: {e}")
    except Exception as e:
        talk(f"An error occurred: {e}")
        print(f"Error: {e}")
    return command

def is_connected():
    """Check if the system is connected to the internet."""
    try:
        requests.get('https://www.google.com', timeout=3)
        return True
    except requests.ConnectionError:
        return False

def get_weather():
    """Get current weather information."""
    api_key = 'your_openweather_api_key'
    base_url = "http://api.openweathermap.org/data/2.5/weather?"
    city_name = "your_city"
    complete_url = base_url + "q=" + city_name + "&appid=" + api_key
    response = requests.get(complete_url)
    if response.status_code == 200:
        data = response.json()
        main = data['main']
        weather_desc = data['weather'][0]['description']
        temperature = round(main['temp'] - 273.15, 2)  # Convert Kelvin to Celsius
        talk(f"The weather in {city_name} is currently {weather_desc} with a temperature of {temperature} degrees Celsius.")
    else:
        talk("Sorry, I couldn't fetch the weather information.")

def get_news():
    """Fetch latest news headlines."""
    api_key = 'your_news_api_key'
    url = f'https://newsapi.org/v2/top-headlines?country=us&apiKey={api_key}'
    response = requests.get(url)
    if response.status_code == 200:
        news_data = response.json()
        articles = news_data['articles'][:5]
        for i, article in enumerate(articles, start=1):
            talk(f"Headline {i}: {article['title']}")
    else:
        talk("Sorry, I couldn't fetch the news.")

def set_alarm(time_str):
    """Set an alarm for the given time."""
    alarm_time = datetime.datetime.strptime(time_str, '%H:%M')
    now = datetime.datetime.now()
    while now < alarm_time:
        now = datetime.datetime.now()
    talk("Time to wake up!")

def send_email(recipient, subject, body):
    """Send an email."""
    sender_email = "your_email@gmail.com"
    sender_password = "your_password"
    message = MIMEMultipart()
    message['From'] = sender_email
    message['To'] = recipient
    message['Subject'] = subject
    message.attach(MIMEText(body, 'plain'))

    try:
        with smtplib.SMTP('smtp.gmail.com', 587) as server:
            server.starttls()
            server.login(sender_email, sender_password)
            server.send_message(message)
        talk("Email has been sent.")
    except Exception as e:
        talk(f"Failed to send email. Error: {e}")
        print(f"Email sending error: {e}")

def run_alexa():
    """Main function to run the assistant."""
    command = take_command()
    if command:
        print(f"Recognized Command: {command}")
        if 'play' in command:
            song = command.replace('play', '').strip()
            talk(f'Playing {song}')
            if is_connected():
                pywhatkit.playonyt(song)
            else:
                talk("Please check your internet connection.")
        elif 'time' in command:
            time = datetime.datetime.now().strftime('%I:%M %p')
            talk(f'Current time is {time}')
        elif 'who the heck is' in command:
            person = command.replace('who the heck is', '').strip()
            info = wikipedia.summary(person, 1)
            talk(info)
        elif 'date' in command:
            talk('sorry, I have a headache')
        elif 'are you single' in command:
            talk('I am in a relationship with wifi')
        elif 'joke' in command:
            talk(pyjokes.get_joke())
        elif 'weather' in command:
            if is_connected():
                get_weather()
            else:
                talk("Please check your internet connection.")
        elif 'news' in command:
            if is_connected():
                get_news()
            else:
                talk("Please check your internet connection.")
        elif 'set alarm for' in command:
            time_str = command.replace('set alarm for', '').strip()
            set_alarm(time_str)
        elif 'send email' in command:
            talk('Who is the recipient?')
            recipient = take_command()
            talk('What is the subject?')
            subject = take_command()
            talk('What is the body of the email?')
            body = take_command()
            send_email(recipient, subject, body)
        else:
            talk('Please say the command again.')
    else:
        talk("No command recognized.")

# Run the assistant
if __name__ == "__main__":
    run_alexa()


ModuleNotFoundError: No module named 'speech_recognition'